# Banc Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
05/07/2024

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os

In [11]:
%matplotlib
%matplotlib

Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg


# Paramètres

In [12]:
# SENSIBILITE CAPTEUR
# S_force_bs =       # bs pour banc statique, sensibilité en N/mV
G_force_bs = 900        # gain du banc de mesure

# cannal d'acquisition
# Cannal 2 

In [13]:
dossier = '2024.05.30'
imag = '/image_'

# Fonctions

In [14]:
def bord_anche(image):
    points = []
    I = np.flipud(image)
    (H, W) = I.shape
    # print(H,W)
    plt.imshow(I, cmap='gray', extent=[0, W, H, 0])
    plt.axis('equal')
    plt.xlim(0, W)
    plt.ylim(0, H)
    plt.title("Cliquez sur les bords de l'anche")
    coord_anche = plt.ginput(2)
    plt.close()
    return coord_anche

In [15]:
# FONCTION DE ROTATION D'IMAGE 
# entrée : une image
# ginput : bord gauche et bord droit du bec
# sortie : angle de rotation, image J après rotation, coordonées des points selectionnés
def rotate(image, coord):
    # Calculer l'angle pour aligner l'image
    (x1, y1), (x2, y2) = coord
    rot_angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
    # Calculer le centre de l'image
    (h, w) = image.shape
    center = (w // 2, h // 2)

    # Créer une matrice de rotation centrale
    rot_mat = cv2.getRotationMatrix2D(center, rot_angle, 1.0)

    # Appliquer la rotation
    img_rot = cv2.warpAffine(image, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
    return rot_mat, img_rot


In [16]:
# ROGNER IMAGE
# entrée : image, coordonées du bec (_,_, xy_bec = im_rotate(J))
def rogner(image, coord):
    # img_rogn = np.flipud(image)
    x_BG = coord[0][0]
    y_BG = coord[0][1] 
    x_BD = coord[1][0]
    y_BD = coord[1][1]
    
        # bornes 
    wmin = int(x_BG)-20     # width min
    wmax = int(x_BD)+20
    hmin = int(y_BG-50)     # height min
    hmax = int(y_BG+350)
    img_rogn = image[hmin:hmax, wmin:wmax] 
    
    return img_rogn

In [20]:
def deplacement_profil(image, seuil, coord):
    # s'assurer que I est bien en binaire
    _, I = cv2.threshold(image, seuil, 255, cv2.THRESH_BINARY)
    
    # coord = bord_anche(image)
        
    (x1, y1), (x2, y2) = map(lambda p: (int(p[0]), int(p[1])), coord)
    # initialisation matrices vides
    Xv = np.arange(x1, x2, dtype=int)   # vecteur de la taille de la largeur de l'image coupée
    Yv = []

# PROBLEME POUR GENERALISER A UNE TRANCHE ENTIERE
    for xv in Xv:
        Iv = I[:, xv].astype(float)   # vecteur qui contient une tranche de l'image largeur 1 (x) et hauteur de l'image (y)
        Iv_2 = np.round(np.gradient(Iv)).astype(int)     # Dérivée centrée (flottante) + Arrondir + convertir en entiers

        # seuillage trance par tranche
        smin = -30
        pos1 = np.where(Iv_2 < smin)[0][-1] if any(Iv_2 < smin) else None
        H = I.shape[0]
        Y = np.arange(1,H+1)
    
        if pos1 is not None:
            Yv.append(Y[pos1])
        else:
            Yv.append(np.nan)
    
    plot_fig = False
    if plot_fig :
        plt.figure()
        plt.imshow(image, cmap='gray')
        plt.plot(Xv, Yv, 'm.-', linewidth=0.1)
            
    return Xv, Yv



In [18]:
seuil = 120
I = cv2.imread(dossier + imag + '0_0.jpg', cv2.IMREAD_GRAYSCALE)
_, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY)
I = I.astype(np.float32)

xy_anche = bord_anche(I)
rot_mat, I_rot = rotate(I, xy_anche)
I_rogn = rogner(I_rot, xy_anche)
coord = bord_anche(I_rogn)
Xv, Yv = deplacement_profil(I_rogn, seuil, coord)

# plt.imshow(I_rogn, cmap='gray')
# print(Xv, Yv)



In [25]:
plt.close('all')
seuil = 120
S_camera = 98  # pix/mm
# Initialisation sur une image

# Lecture de l'image en nuance de gris, mise en noir et blanc
I = cv2.imread(dossier + imag + '0_0.jpg', cv2.IMREAD_GRAYSCALE) # commencer par l'image de la poutre rigide définissant l'horizontalité
# I = cv2.imread('a2_test.bmp', cv2.IMREAD_GRAYSCALE)
_, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY)
I = I.astype(np.float32)

xy_anche = bord_anche(I)
# rot_mat, I_rot = rotate(I, xy_anche) # pas besoin de faire une rotation 
I_rogn = rogner(I, xy_anche)
coord_rogn = bord_anche(I_rogn)
print(coord_rogn)

Ys = 1
Zs = 20
pas_z = 0.05 #mm
(w, h) = I_rogn.shape


plot_fig = True
# if plot_fig :
#         fig, axs = plt.subplots(1, 2, figsize=(12, 4))
        
# Import data
for y in range(Ys):
    if plot_fig :
            fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    with open(dossier + f'/profil{y}.txt', 'w') as file:
        for z in range(Zs):
            img_path = dossier + imag + f'{y}_{z}.jpg'
            # img_path = 'a2_test.bmp'
            if not os.path.exists(img_path):
                continue
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            _, img = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)
            img = img.astype(np.float32)
            # _, img_rot = rotate(img, xy_anche)
            img_rogn =  rogner(img, xy_anche)
            

            Xv, Yv = deplacement_profil(img_rogn, seuil, coord_rogn)
            Yv_array = np.array(Yv)
            
            # se débarasser des nan
            mask = ~np.isnan(Xv) & ~np.isnan(Yv_array) & ~np.isinf(Xv) & ~np.isinf(Yv_array)
            Xv_clean = Xv[mask]
            Yv_clean = Yv_array[mask]
            
            # grandeur physique
            Xv_physique = Xv_clean/S_camera
            Yv_physique = Yv_clean/S_camera
        
            # polyfit
            coefficients_fit = np.polyfit(Xv_physique, Yv_physique, 3)
            
            # traitement pour faire plus que le polyfit 
        
            if plot_fig :
                axs[0].imshow(img_rogn, cmap='gray')
                axs[0].plot(Xv_physique, Yv_physique, '-', lw=2)
                axs[0].set_xlabel('y (mm)')
                axs[0].set_xlabel('z (mm)')
                axs[1].plot(Xv_physique, -np.polyval(coefficients_fit, Xv_physique), lw=2)
                axs[1].plot(Xv_physique, -Yv_physique, lw=2)
                axs[1].grid()
                

            file.write(f'{y} {coefficients_fit[0]} {coefficients_fit[1]} {coefficients_fit[2]} \n')
    

In [75]:
plt.close('all')
seuil = 120
S_camera = 98  # pix/mm
# Initialisation sur une image

# Lecture de l'image en nuance de gris, mise en noir et blanc
I = cv2.imread(dossier + imag + '0_0.jpg', cv2.IMREAD_GRAYSCALE) # commencer par l'image de la poutre rigide définissant l'horizontalité
# I = cv2.imread('a2_test.bmp', cv2.IMREAD_GRAYSCALE)
_, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY)
I = I.astype(np.float32)

xy_anche = bord_anche(I)
# rot_mat, I_rot = rotate(I, xy_anche) # pas besoin de faire une rotation 
I_rogn = rogner(I, xy_anche)
coord_rogn = bord_anche(I_rogn)
# print(coord_rogn)

Ys = 5
Zs = 20
pas_z = 0.05 #mm
(w, h) = I_rogn.shape


plot_fig = True
# if plot_fig :
#         fig, axs = plt.subplots(1, 2, figsize=(12, 4))
        
# Import data
for y in range(Ys):
    if plot_fig :
            fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    with open(dossier + f'/profil{y}.txt', 'w') as file:
        for z in range(Zs):
            img_path = dossier + imag + f'{y}_{z}.jpg' # recherhcer toutes les images
            if not os.path.exists(img_path):        # si le chemin n'est pas trouvé, cette image est ignorée et on passe à la suivante
                continue
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)    # lecture d'image en niveau de gris
            _, img = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)  # mise en noir et blanc selon le seuil défini
            img = img.astype(np.float32)
            # _, img_rot = rotate(img, xy_anche)
            img_rogn =  rogner(img, xy_anche)       # rogner l'image proche de l'anche
            # print(img_rogn)
            
            Xv, Yv = deplacement_profil(img_rogn, seuil, coord_rogn) # détection de la pointe de l'anche image par image
            Yv_array = np.array(Yv)
            # se débarasser des valeurs abérentes (nan dans les Xv et Yv)
            mask = ~np.isnan(Xv) & ~np.isnan(Yv_array) & ~np.isinf(Xv) & ~np.isinf(Yv_array)
            Xv_clean = Xv[mask]
            Yv_clean = Yv_array[mask]
            # grandeur physique : on passe se pixel en mm
            Xv_physique = Xv_clean/S_camera
            Yv_physique = Yv_clean/S_camera
                
            # Etude de notre pointe par anche
            Xv_slice_center = []
            Yv_mean_per_slice = []
            nbr_slice = 20
            slice_size = (np.abs(Xv_clean[-1]-Xv_clean[0]))/nbr_slice   # taille en pixel d'une tranche
            for n in range(nbr_slice-1):
                # début et fin de chaque slice par itération            
                slice_left = int(Xv_clean[0]+n*slice_size)
                slice_right = int(Xv_clean[0]+(n+1)*slice_size)
                Xv_slice_center.append(slice_left + np.abs(slice_left-slice_right)/2) # coordonnée centrale de la tranche
                Yv_slice = Yv_clean[slice_left:slice_right]
                Yv_mean_per_slice.append(np.mean(Yv_slice)) # déplacement moyen par slice (moyenne sur les Yv)
            
            # grandeur physique, mm
            Xv_slice_center_phys = np.array(Xv_slice_center)/S_camera
            Yv_mean_per_slice_phy = np.array(Yv_mean_per_slice)/S_camera

            # Etude des polyfit sur données totales et données par tranches optionnelle
            # coefficients_fit = np.polyfit(Xv_physique, Yv_physique, 3)
            # coefficients_fit_slice = np.polyfit(Xv_slice_center_phys, Yv_mean_per_slice_phy, 3)
            # diff = coefficients_fit-coefficients_fit_slice
            # rmse = np.sqrt(np.mean((diff) ** 2))
            # print(rmse)
            
            
            # traitement pour faire plus que le polyfit 
        
            if plot_fig :
                axs[0].imshow(img_rogn, cmap='gray')
                axs[0].plot(Xv_physique, Yv_physique, '-', lw=2)
                axs[0].set_xlabel('y (mm)')
                axs[0].set_xlabel('z (mm)')
                # axs[1].plot(Xv_physique, -np.polyval(coefficients_fit, Xv_physique), '--', lw=2)
                # axs[1].plot(Xv_slice_center_phys, -np.polyval(coefficients_fit_slice, Xv_slice_center_phys), lw=2)
                axs[1].plot(Xv_slice_center_phys, -Yv_mean_per_slice_phy, '.-', lw=2)
                axs[1].grid()
                

            file.write(f'{y} {coefficients_fit[0]} {coefficients_fit[1]} {coefficients_fit[2]} \n')
    